# MapReduce

MapReduce is a popular paradigm for distributed computing on big data. While it has been replaced by generalized dataflow systems like [Apache Spark](https://spark.apache.org/) and [Microsoft Dryad](https://www.microsoft.com/en-us/research/project/dryad/), it is a very simple concept that lends itself to distributed computing.

Here you will implement a simple version of MapReduce and use it to solve problems.

In [1]:

import re

import multiprocessing
import time
from typing import TypeVar, Hashable, Dict, List, Callable, Iterable, Tuple, Optional
import collections
import tarfile
import gzip
import itertools
from urllib.parse import urlparse

import nltk
from nltk.corpus import gutenberg
from nltk.corpus import stopwords

from bs4 import BeautifulSoup

from testing.testing import test

def nltk_download_test(nltk_download):
    nltk_download()

@test
def nltk_download():
    nltk.download('stopwords')
    nltk.download('gutenberg')
    

### TESTING nltk_download: PASSED 0/0
###



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lilianghui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/lilianghui/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


## MapReduce Implementation

MapReduce is conceptually very simple. Read the lecture notes, and optionally the [Wikipedia Entry](https://en.wikipedia.org/wiki/MapReduce) to understand it. Here's a summary:

1. The Map function is called on each element of a list of inputs, producing a list of dictionaries. Each input (in any format) is converted to a dictionary from a key to some intermediate value. Once you figure out what a good intermediate value for the problem you're trying to solve, the Map and Reduce functions are usually obvious. 
2. The Partition function (sometimes called Shuffle or Collate) converts the produced list-of-dictionaries into a dictionary-of-lists. Each key/value pair in the output dictionay is sometimes called a Partition.
3. The Reduce function converts each value in the dictionary-of-lists into a single value, making it a dictionary-of-values.

Now you have to complete this implementation of MapReduce by filling in the partition function. We have included type hints, but you don't need them to complete this; you can optionally read more about them [here](https://mypy.readthedocs.io/en/latest/cheat_sheet_py3.html).

In [2]:
# Type variables for typechecking:
# This is a way to denote the input and output types of mapreduce and various functions.
# We also describe this information in the mapreduce docstring below

TInput = TypeVar("TInput")             # Type of input
TKey = TypeVar("TKey", bound=Hashable) # Key type, must be hashable 
TIntermediate = TypeVar("TIntermediate") # Intermediate representation
TOutput = TypeVar("TOutput")           # The produced output type

def partition_test(partition):
    test.equal(partition([{"A": "A", "B": "B"}, {"B": "B", "C": "C"}]),
              {"A": ["A"], "B": ["B", "B"], "C": ["C"]})

@test
def partition(intermediates : Iterable[Dict[TKey, TIntermediate]]) -> Dict[TKey, List[TIntermediate]]:
        """Organize the mapped values by their key.

        args:
            intermediates : Iterable[Dict[TKey, TIntermediate]] -- a list of dictionaries produced by the mapper

        returns : Dict[TKey, List[TIntermediate]] -- the values in intermediates grouped by key 
        """
        # change the input into tuple
        tuple_list = []
        for intermediate in intermediates:
            for key, value in intermediate.items():
                tuple_list.append((key,value))
        rv = collections.defaultdict(list)
#         print(tuple_list)
        for r,v in tuple_list:
            rv[r].append(v)
#         print(rv)
        return rv

### TESTING partition: PASSED 1/1
###



Once you have done that, the MapReduce implementation is quite simple:

In [3]:
# This is a fake multiprocessing.Pool, used to provide single-thread execution
# of map. You get better error messages with this.
class FakePool():
    def map(self, lmbd, data, chunksize=None):
        return list([lmbd(x) for x in data])
    def __enter__(self):
        return self
    def __exit__(self, a1, a2, a3):
        pass

In [4]:
def mapreduce(mapper  : Callable[[TInput], Dict[TKey, TIntermediate]],
              reducer : Callable[[Tuple[TKey, List[TIntermediate]]], TOutput],
              data    : Iterable[TInput],
              chunksize_map=1, chunksize_reduce=8, pool=None) -> Dict[TKey, TOutput]:
    """ MapReduce. Map the data using mapper, partition it, and reduce each partition using the reducer.
    
    args:
        mapper  : Callable[[TInput], Dict[TKey, TIntermediate]]
                -- a function that takes in an input element and breaks it out into a dictionary from some key
                   to some intermediate value
        reducer : Callable[[Tuple[TKey, List[TIntermediate]]], TOutput]
                -- a function that takes in a tuple of the key and the list of intermediate values and converts
                   it to a single output value; this conversion should not rely on the key
        data    : Iterable[TInput] -- the input data to map over

    kwargs:
        chunksize_map    : int -- an internal parameter used for performance-tuning
        chunksize_reduce : int -- an internal parameter used for performance-tuning
        pool : multiprocessing.Pool -- the pool providing mapping

    returns : List[TOutput] -- the output of running the mapper and reducer on data.
    """
    if pool is None:
        pool = FakePool()
    
    # Isn't the implementation really simple?
    intermediates = pool.map(mapper, data, chunksize_map)
    partitions = partition(intermediates)
    return pool.map(reducer, partitions.items(), chunksize_reduce)

Note that we're using the Python multiprocessing library to split the computation across the cores in your computer. There are [some limitations](https://codewithoutrules.com/2018/09/04/python-multiprocessing/) to this: in particular, you cannot use anonymous functions (i.e. `lambda` expressions). If your Python kernel crashes, you may need to reset it.

Great! Now let's apply this to a very simple example:

## Character Count

A classic example of a MapReduce application is in the counting of words in a corpus; here we count _characters_ instead. The map function converts each documentation into the count of words that appear in the document, and the reduce function adds the counts for each word up, producing a tuple as the output type. The keys are words, and the intermediate representation is the count in each string, and the output is a tuple of character and count.

Now, go through the code below and make sure you understand how to use MapReduce:

In [5]:
def cc_map(data : str) -> collections.Counter:
    """ Read a string and convert it into character frequency dictionary

    args:
        data : str -- the input string
    
    returns : Counter[str, int] -- a dictionary of counts, from each character to the number of times it appears
    """
    return collections.Counter(data)
 
def cc_reduce(t : Tuple[str, List[int]]):
    """ Find the total number of times a character appears in the input from a list of appearances in each file
    
    args:
        t : Tuple[key, frequencies] -- a tuple of
            key : str -- the character
            frequencies : List[int] -- the number of times it appears in each file
    
    returns: Tuple[str, int] -- a tuple of (word, total number of occurrences)
    """
    key, values = t
    return (key, sum(values))

In [6]:
def count_characters_test(count_characters):
    cc = count_characters([nltk.corpus.gutenberg.raw(f) for f in nltk.corpus.gutenberg.fileids()])
    cc.sort(key=lambda x: x[1], reverse=True)
    test.equal(cc[:8], [(' ', 2000723), ('e', 1108892), ('t', 800450), ('a', 698889), ('o', 662391), ('h', 635854), ('n', 607945), ('s', 540033)])    
    # The most common letter is the space character, followed by the letter 'e'.
    print(cc[:8])

@test
def count_characters(alos):
    """ Count the number of times each character appears in the input data
    
    args:
      alos : List[str] -- a list of strings
    """
    
    return mapreduce(cc_map, cc_reduce, alos)

[(' ', 2000723), ('e', 1108892), ('t', 800450), ('a', 698889), ('o', 662391), ('h', 635854), ('n', 607945), ('s', 540033)]
### TESTING count_characters: PASSED 1/1
###



Now let's kick it up a notch with the poster child use-case for MapReduce.

## Inverted Index

An inverted index is a mapping (`dict`) of elements to the list of locations at which the element occurs. When Google released their [2004 MapReduce paper](https://static.googleusercontent.com/media/research.google.com/en//archive/mapreduce-osdi04.pdf), this was one of the use-cases they cited.

The concept of an inverted index is central to any modern search engine. E.g., given a query "X, Y", where, we want to retrieve documents containing the words X or/and Y, the engine only needs to look at the union/intersection of the documents in the inverted indices of X and Y (which have been precomputed already)!

In our version of this, we will be constructing an inverted index for words in documents in the Project Gutenberg corpus.

### Specification

The types in this problem are:

 - The input type is `Tuple[str, List[str]]`, corresponding to the file name and a list of words
 - The key type is `str`, corresponding to each word
 - The intermediate representation type is `List[Tuple[str, int]]`, which is a list of (filename, position) pairs
 - The output type is `Tuple[str, TIntermediate]`, which is a tuple of word and the intermediate representations, with the added constraint that the intermediate representations are in sorted order.

(To make this easier, we've told you what the intermediate representation is -- when writing MapReduce problems, finding the correct intermediate representation is often the challenge.)

In the map step, your output should not include any stopwords in `nltk.corpus.stopwords.words('english')`. You do not need to lower-case the input or lemmatize words, etc.

In [7]:
def ii_map(data : Tuple[str, List[str]]) -> Dict[str, List[Tuple[str, int]]]:
    """ Inverted index map; compile a list of occurrences of each (non-stopword) word in a document.

    args:
        data : Tuple[filename, words] -- the input data, where:
            filename : str -- the filename to tag each word in the index with
            words : List[str] -- the list of words in the document, in the order they appear

    returns : Dict[str, List[Tuple[str, int]] -- the mapped dictionary, where each word has a corresponding list
        of tuples, each with the filename and index at which it appears.
    """
    sw = set(stopwords.words('english'))
    filename, words = data
    rdict = {}
    count = 0
    test = 0
    for w in words:
        if w not in sw:
            if w not in rdict.keys():
                rdict[w] = [(filename,count)]
            else:
                rdict[w] = rdict[w] + [(filename,count)]      
        count += 1
    return rdict

def ii_reduce(t : Tuple[str, List[List[Tuple[str, int]]]]):
    """ Assemble and sort the inverted index.

    args:
        t : Tuple[key, List[occurrences]] -- a tuple of
            key : str -- the word 
            occurrences : List[Tuple[str, int]] -- the file name and word number in the file

    returns : Tuple[key, occurrences] -- a tuple of
            key : str -- the word 
            occurrences : List[Tuple[str, int]] -- the occurrences, flattened and sorted

    """
    key, occurrences = t
    # flatten the result

    if type(occurrences[0]) == list:
        occurrences = [item for sublist in occurrences for item in sublist]

    occurrences = (sorted(occurrences,key = lambda test: test))
    return key,occurrences

In [8]:
def inverted_index_test(inverted_index):
    tt = inverted_index([("@", list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")), ("%", list("WXYZ"))])
    test.equal(tt[:4], [('A', [('@', 0)]), ('B', [('@', 1)]), ('C', [('@', 2)]), ('D', [('@', 3)])])
    test.equal(tt[-2], ('Y', [('%', 2), ('@', 24)]))
    test.equal(tt[-1], ('Z', [('%', 3), ('@', 25)]))

    books = ['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt']
    gutenberg = inverted_index([(f.replace(".txt", "").replace("austen-", ""), list(nltk.corpus.gutenberg.words(f))) for f in books])
    g = dict(gutenberg)

    # Check that you have all entries for some words:
    test.equal(len(g[","]), 27601)
    test.equal(len(g["I"]), 6306)
    test.equal(len(g["could"]), 1837)
    test.equal(len(g["Mr"]), 1587)
    test.equal(len(g["Emma"]), 866)

    # Check output for some words:
    test.equal(g["Austen"], [('emma', 4), ('persuasion', 4), ('sense', 6)])
    # If this works but the next test fails, then you aren't ordering the 
    test.equal(set(g["likelihood"]), set([('emma', 5715), ('emma', 110230), ('persuasion', 68360), ('sense', 83037), ('sense', 120041)]))
    test.equal(g["likelihood"], [('emma', 5715), ('emma', 110230), ('persuasion', 68360), ('sense', 83037), ('sense', 120041)])
    
    # Check which files the name "Emma" is mentioned in:
    test.equal(collections.Counter(v for v, _ in g["Emma"]), {"emma": 865, "persuasion": 1})

@test
def inverted_index(data):
    """ Count the number of times each character appears in the input data
    
    args:
      data : Tuple[str, List[str]] -- a list of (filename, words)
    """
    return mapreduce(ii_map, ii_reduce, data)

### TESTING inverted_index: PASSED 12/12
###



## Reverse Web Graph

The world-wide web can be throught of as a directed graph, where the edge A → B encodes a link in page A that points to page B. We've previously covered adjacency matrices in this course; we can very easily construct a row in the adjacency matrix for each page by parsing each webpage and recording the outgoing links we observe in it.

Now we want the *reverse* adjacency list. For each webpage, we want the list of webpages which point to it (i.e., incoming links). This is the reverse webgraph problem, and we can solve this with MapReduce:

### Specification

The types in this problem are:

 - The input type is `Tuple[str, bytes]`, corresponding to the URL and the raw bytes of the website
 - The key type is `Tuple[str, str]`, corresponding to the URL `netloc` and `path`. We explain these parts in the Hints section.
 - The intermediate type is up to you!
 - The output type is `Tuple[key, Dict[incoming_netloc, incoming_path]]`, where:
     - `key` is a `Tuple[str, str]`, same as the key type.
     - `incoming_netloc` is a `str` which has the `netloc` of the incoming links, and
     - `incoming_path` is a `Set[str]` which is a set of `path`s following the associated `netloc`.

Details:

- When constructing the index, we do not care how many links point from A to B.
- You can choose any way to represent the data.
- You should use `BeautifulSoup` to parse the webpage.
    - Only search for URLs in tags like `<a href="...">`.
    - Only consider URLs ending in `.html`
- URLs can be relative or absolute; you should use [`urlparse`](https://docs.python.org/3/library/urllib.parse.html#urllib.parse.urlparse) to extract the `netloc` and `path` to determine which path this is.
    - If the URL is absolute (such as `http://cs.cornell.edu/Info/Courses/Current/CS415/CS414.html`), then `netloc` will not be blank. You can use the parsed `netloc` and `path` directly.
    - If the URL is relative (such as `/Info/Courses/Current/CS415/CS414.html` or `CS414.html`), then `netloc` will be blank. You can assume the `netloc` of the target is the same as the `netloc` of the source, and:
        - If the `path` has a leading `/`, then it is the complete path of the target.
        - If the `path` does not have a leading `/`, you should replace the final path segment (i.e. everything after the final `/`) with the contents of `path`. You do not need to consider parent paths (i.e. `../`).
            - if the `path` is empty, then you should replace the entire path with the target `path`, adding a leading `/`.
- Not all target pages may appear in the source; this is alright.
- Ignore the `Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.` message.

The most fiddly part of this problem is in handling URLs; we provide you with the stub `canonicalize_url` and some tests which you can optionally use.

In [9]:
def canonicalize_url_test(canonicalize_url):
    source = ("source.edu", "/path/segment/leaf.html")
    test.equal(canonicalize_url(("evenbug.edu", "/a.html"), "file:///c.html"), ("evenbug.edu", "/c.html"))
    test.equal(canonicalize_url(source, "https://a.edu/research/test.html"), ("a.edu", "/research/test.html"))
    test.equal(canonicalize_url(source, "http://a.edu/research/test.html"), ("a.edu", "/research/test.html"))
    test.equal(canonicalize_url(source, "/research/test.html"), ("source.edu", "/research/test.html"))
    test.equal(canonicalize_url(source, "research/test.html"), ("source.edu", "/path/segment/research/test.html"))
    test.equal(canonicalize_url(source, "test.html"), ("source.edu", "/path/segment/test.html"))
    # You should pretend .. is a folder name:
    test.equal(canonicalize_url(source, "../test.html"), ("source.edu", "/path/segment/../test.html"))
    test.equal(canonicalize_url(source, "test.pdf"), None)
    # This clarifies an edge case:
    test.equal(canonicalize_url(("oddbug.edu", ""), "test.html"), ("oddbug.edu", "/test.html"))
    test.equal(canonicalize_url(("oddbug.edu", ""), "a/b/test.html"), ("oddbug.edu", "/a/b/test.html"))

@test
def canonicalize_url(source : Tuple[str, str], target_str : str) -> Optional[Tuple[str, str]]:
    """Given a source (netloc, path) and a target (netloc, path), construct the canonical target (netloc, path)

    args:
        source : Tuple[str, str] -- source (netloc, path)
        target_str : str -- target url, not canonical

    returns : Optional[Tuple[str, str]] -- canonical target (netloc, path) or None if the URL should be ignored
    """
    # parse target
    target = urlparse(target_str)
    # netloc
    if target.netloc != "":
        netloc = target.netloc
    else:
        netloc = source[0]
    
    # path

    # absolute path
    if target.path == "":
        path = '/' + source[1]
    elif len(target.path) >= 1:
        if target.path[0] == '/':
            # end with .html
            if len(target.path) >5 and (target.path[-5:] == ".html"):
                path = target.path
            else:
                return None
        # relative path
        else:
            if len(target.path) >5 and (target.path[-5:] == ".html"):
                path = target.path
                # corner case
                if source[1] == "":
                    path = '/' + target.path
                else:
                    # match the source path
                    test1 = re.compile(".*\/")
                    prev = test1.match(source[1])
                    path = prev.group() + target.path
            else:
                return None
    else:
        print(target_str)
        path = ""

    return netloc, path

print(canonicalize_url(("",""),"file://issl.cs.byu.edu/docs/CS556/Syllabus95.html"))

### TESTING canonicalize_url: PASSED 10/10
###

('issl.cs.byu.edu', '/docs/CS556/Syllabus95.html')


In [10]:
def rwg_map(datum):
    """populate the (target, source) dict for every hyperlink in the file.

    args:
        datum : Tuple[str, bytes] -- Tuple of URL and page content

    returns : Dict[target, source] -- where
        target : Tuple[str, str] -- the netloc, path of the destination webpage
        source : YourIntermediateType -- the source webpage, as your chosen type.
    """
    # href represents that what website it is pointing to
    url, html = datum
    soup = BeautifulSoup(html,'html.parser')
    url_dict = {}
    # find all a href, and create the dictionary as prev
    for i in soup.find_all('a'):
        # get source and target
        source = canonicalize_url(("",""),url)
        if i.get('href') == None:
            print("i.get('href') is none here")
        target = canonicalize_url(source,i.get('href'))
        # set target as the key, source as the result
        if target and source:
            if target not in url_dict.keys():
                url_dict[target] = [source]
            else:
                url_dict[target] = url_dict[target] + [source]
    return url_dict

def rwg_reduce(item):
    """Restructure the gathered backlinks

    args:
        item : Tuple[target, sources] -- where
            target : Tuple[str, str] -- is the (netloc, path) of the target webpage
            sources : List[YourIntermediateType] -- is the list of representations of source webpages

    returns: Tuple[key, Dict[incoming_netloc, incoming_path]] -- where
        key : Tuple[str, str] -- same as the key type
        incoming_netloc : str -- which has the `netloc` of the incoming links
        incoming_path : Set[str] -- which is a set of `path`s that follow the associated `netloc`
    """
    target, sources = item
    # flatten sources
    if type(sources[0]) == list:
        sources = [item for sublist in sources for item in sublist]
    
    # sort incoming netloc, and create set of strings
    rdic = {}
    for i in sources:
        incoming_netloc = i[0]
        incoming_path = i[1]
        if incoming_netloc not in rdic.keys():
            rdic[incoming_netloc] = [incoming_path]
        else:
            rdic[incoming_netloc] = rdic[incoming_netloc] + [incoming_path]
    
    # get the set of rdic
    for k,v in rdic.items():
        rdic[k] = set(v)
    return target, rdic

# test map function and reduce function
# rwg_map(datum)
print(canonicalize_url(("",""),"file://issl.cs.byu.edu/docs/CS556/Syllabus95.html"))

('issl.cs.byu.edu', '/docs/CS556/Syllabus95.html')


In [11]:
def read_webpages():
    tar = tarfile.open("webgraph.tar.gz", "r:gz")
    data : List[Tuple[str, bytes]] = []
    for member in tar.getmembers():        
        num = member.name[:-5].split("_")[-1]
        try:
            if int(num) < 1000:
                f = tar.extractfile(member)
                if f is not None:
                    # The first line is the URL, all subsequent lines are the data:
                    website : str = next(f).decode("utf-8").strip()
                    content : bytes = f.read()
                    data.append((website, content))
        except ValueError:
            pass

    return data

def reverse_webgraph_test(reverse_webgraph):
    data = read_webpages()

    # Here we use a multiprocessing.Pool, which parallelizes execution over multiple cores.
    with multiprocessing.Pool() as pool:
        rw = dict(reverse_webgraph(data, pool=pool))

    #l = [sorted(((k, len(v)) for k, v in rw.items() if len(v) > 1), key=lambda x: x[1])]
    v = {}
    for k in [
    ('www.cs.byu.edu', '/byu.html'),
    ('www.cs.berkeley.edu', '/~russell/aima.html'),
    ('uu-gna.mit.edu:8001', '/uu-gna/text/cc/index.html'),
    ('www.ncsa.uiuc.edu', '/General/Internet/WWW/HTMLPrimer.html'),
    ('www.cs.umass.edu', '/rcfdocs/newhome/index.html')]:
        v[k] = rw[k]
        
    print(v)
    
@test
def reverse_webgraph(data, pool=None):
    """ Count the number of times each character appears in the input data
    
    args:
      data : Tuple[str, List[str]] -- a list of (filename, words)
    """
    return mapreduce(rwg_map, rwg_reduce, data, pool=pool)

i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
None
None
None
i.get('href') is none here
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
None
None
i.get('href') is none here
i.get('href') is none here
i.get('href') is none here
i.get('href') is none here
None
None
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
i.get('href') is none here
None
i.get('href') is none here
None
None
None
i.get('href') is none here
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is n

None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
i.get('href') is none here
None
i.get('href') is none here
None
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
None
None
i.get('href') is none here
i.get('href') is none here
i.get('href') is none here
None
None
None
i.get('href') is none here
i.get('href') is none here
i.get('href') is none here
None
None
None
i.get('href') is none here
i.get('href') is none here
i.get('href') is none here
None
None
None
i.get('href') is none here
i.get('href') is none here
None
i.get('h

i.get('href') is none here
i.get('href') is none here
i.get('href') is none here
None
None
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('h

i.get('href') is none here
None
None
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
None
i.get('href') is none here
i.get('href') is none here
None
None
i.get('href') is none here
Non

## PageRank

MapReduce can also be used to perform the update step in any iterative computation, such as PageRank. In this assignment we're writing a pagerank that splits the pagerank algorithm into batches of columns.

We will try that on the same graphs you may have used back in Homework 2. For your reference, the simple graph is this:
```
    A -> B -> C
    |    ^
    v    |
    D -> E
``` 

The larger test graph is from Wikipedia.

You are allowed to consult online sources as long as all code remains your own. Bear in mind that a lot of online sources involve making PageRank efficient in matrices so large that no single row or column can be held in memory. 

### Algorithm

We have discussed the PageRank algorithm in class; here is some naive pseudocode for calculating it:

```python
nodes = [...] # A collection of nodes
edges = [...] # A collection of edges

pr = {...} # Uniform initialization of pagerank
d = 0.85   # Smoothing
for i in range(iters):
    update = {...} # Zeros, one for each node
    
    # create the initial updating matrix
    ###### Mapping function #########
    for source in nodes:
        # Count the number of outgoing edges from source
        num_outgoing = len(e for e in edges if e is (source -> *))
        
        # If this is a dead-end node:
        if num_outgoing == 0:
            for destination in nodes:
                update[destination] += pr[source]/len(nodes)
        # This node has outgoing connections:
        else:
            for destination in nodes:
                if (source -> destination) in edges:
                    update[destination] += pr[source]/num_outgoing
    
    # Actually calculating the value
    # Update with smoothing:
    for node in nodes:
        pr[node] = update[node]*d + (1-d)/n
```

Note that you should not actually use this algorithm for calculating this; that would be too slow. Instead, you should:

1. Observe that your function is given `num_outgoing`, the number of outgoing edges from each node, and `incoming`, the source nodes that have edges ending in your nodes.
2. Rewrite the above pseudocode to calculate the new pagerank for a single node using `incoming` rather than as shown in the pseudocode above.
3. Implement this using PageRank (Additional hint: the reduce function is very simple! All the heavy lifting is done in the map function.)
4. Verify that your algorithm is numerically correct on the small, and then the large test case provided.
5. Time your algorithm and make sure it completes all tests in under 120s. The Optimization section has hints on this.

It may be a good idea to come to TA hours with pseudocode or ideas.

### Optimization

One of the challenges is making your code run quickly. We've already arranged the code so you should be able to meet the time requirement. Also, we will be evaluating your code on only a few iterations instead of 100 iterations. Ensure that your function takes under 120s on a modern laptop.

Here are some hints on optimization:

1. While you have sufficient information to solve the problem with the arguments to `pr_map`, you can significantly improve the speed by precomputing a little information. Feel free to change `gen` in `pagerank` to pass precomputed information to the workers.
2. It may not be at all necessary to improve this, though, so submit your code once before trying to optimize it. 
3. We use the generator `gen` to bundle arguments to feed them to `mapreduce`. We are grouping the incoming nodes into chunks of `2**11 = 2048` nodes; we experimentally found that this balances the overhead of transferring data to the worker processes against the amount of work per chunk. You may change this. 


You will notice that this method is much slower than that in Homework 2. It took about 6s to run all tests using the original sparse matrix implementation on a single core; this will take about a 60s to do the same on a 6-core machine. The main reasons for the slow-down are the setup time (in spawning new processes for the multiprocessing pool), the communication overhead (marshaling, sending data to each process, and unmarshaling it), and the use of the Python dictionary instead of heavily-optimized Numpy sparse matrix math.

### Specification

You should complete `pr_map` and `pr_reduce` (and optionally modify `pagerank`) to calculate the pagerank of nodes in the input graph, following the type signatures. 100 iterations of this should complete in under 120s. You may choose any intermediate representation (though a very simple one is sufficient to solve the problem).

In [14]:
# Utility function to read the edges:
def read_graph(basename="wikipedia_small"):
    nodes = []
    num_outgoing = {}
    incoming = {}
    with gzip.open(f"{basename}.nodes.gz", 'rt', encoding="utf-8", newline="") as f:
        for a in f:
            a = a.strip()
            nodes.append(a)
            num_outgoing[a] = 0
            incoming[a] = []

    with gzip.open(f"{basename}.graph.gz", 'rt', encoding="utf-8", newline="") as f:
        links = []
        for row in f:
            i, j = tuple(row.strip().split())
            num_outgoing[nodes[int(i)]] += 1
            incoming[nodes[int(j)]].append(nodes[int(i)])
    return num_outgoing, incoming

# Utility function to help group the input into chunks:
# https://docs.python.org/3/library/itertools.html
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
    args = [iter(iterable)] * n
    return itertools.zip_longest(fillvalue=fillvalue, *args)

In [15]:
def pagerank_test(pagerank):
    G_num_outgoing = {"A": 2, "B": 1, "C": 0, "D": 1, "E": 1}
    G_incoming = { "A": set(), "B": set("AE"), "C": set("B"), "D": set("A"), "E": set("D") }

    pr = pagerank(G_num_outgoing, G_incoming)    
    test.true(abs(pr['A']-0.08510862387068166) < 1e-7)
    test.true(abs(pr['B']-0.28124676686965944) < 1e-7)
    test.true(abs(pr['C']-0.3241683757098922) < 1e-7)
    test.true(abs(pr['D']-0.12127978901572137) < 1e-7)
    test.true(abs(pr['E']-0.18819644453404483) < 1e-7)
    
    # Remove this line to run the rest of the tests:
    GW_num_outgoing, GW_incoming = read_graph()
    
    with multiprocessing.Pool() as pool:
        start_time = time.perf_counter()
        pr = pagerank(GW_num_outgoing, GW_incoming, pool=pool, iters=100)
        duration = time.perf_counter() - start_time
        
    test.equal(len(pr), 24166)

    # Numerical check
    test.true(abs(pr['United_States'] - 0.00275188705264) < 1e-5)
    test.true(abs(pr['2008']          - 0.00217342514773) < 1e-5)
    test.true(abs(pr['Canada']        - 0.00109896195215) < 1e-5)
    test.true(abs(pr['World_War_II']  - 0.00104913079624) < 1e-5)
    test.true(abs(pr['List_of_African_films'] - 0.00100713870383) < 1e-5)
    test.true(abs(pr['Europe']        - 0.000937690025073) < 1e-5)
    test.true(abs(pr['English_language'] - 0.000908144359626) < 1e-5)
    test.true(abs(pr['Geographic_coordinate_system'] - 0.000891711151403) < 1e-5)
    test.true(abs(pr['Latin']         - 0.000888662228804) < 1e-5)

    print(f"Completed pagerank in {round(duration, 1)}s")
    test.true(duration < 120)

In [16]:
# this process is not dynamic, but just can be done parallel
def pr_map(inp):
    """PageRank map function
    
    args:
        inp: Tuple[entries, pr, num_outgoing, d,...] -- where
            entries : List[Tuple[node, incoming]] -- a list of nodes to process, where
                node : str -- the node id
                incoming : Collection[str] -- the node ids that have an edge to this node
            pr : Dict[str, float] -- from node id to pagerank at the start of the iteration
            num_outgoing : Dict[str, int] -- from node id to the number of edges leaving that node
            d : float -- the damping factor
            ...additional entries to speed up your code
            zeros_list: list all zeros nodes, update rv at once.
    
    returns : Dict[str, Intermediate], where the key is the node id and Intermediate is whatever your
            intermediate step representation is. Each entry in entries must be represented in your output
    """
    entries, pr, num_outgoing, d, zeros_list= inp

    rv = {}
    n = len(num_outgoing.keys())
    nodes = num_outgoing.keys()
    
    # cal update value
    update = 0
    
    for node,_ in filter(lambda x: x, entries):
        if node in zeros_list:
            update += pr[node] / n
    for node in num_outgoing.keys():
        rv[node] = update
        
    # We remove null entries created by grouper:
    for dest, incoming_links in filter(lambda x: x, entries):
        for source in incoming_links:
            rv[dest] += pr[source] / num_outgoing[source]
            
    for node in num_outgoing.keys():
        rv[node] = (d,n,rv[node])
    return rv

def pr_reduce(inp):
    """PageRank reduce function
    
    args:
        inp : Tuple[str, List[Intermediate]]
        
    returns: Tuple[str, float] -- the node id and its weight at the end of this iteration
    """
    node, list_of_reps = inp
    # special case A, with not incoming nodes 
    # apply iteration of optimization
    pr = 0
#     print(list_of_reps)
    for i in list_of_reps:
        pr += i[2]
    d = list_of_reps[0][0]
    n = list_of_reps[0][1]
    num = (1 - d)/n + d * pr
    return node, num

@test
def pagerank(num_outgoing, incoming, d=0.85, iters=100, pool=None):
    """ Compute the PageRank score for each node in the network using a MapReduce version of the power method

    args:
        num_outgoing: Dict[str, int] -- the number of outgoing links from each node
        incoming: Dict[str, Set[str]] -- the set of pages that link to each node

    kwargs:
        d : float -- damping factor (note that this is defined the same as homework 2)
        iters : int -- number of iterations

    returns: Dict[str, int] -- the importance score for each node
    """
    n = len(num_outgoing)
    # Initialize the pagerank as a uniform distribution. You should not change this:
    pr = {k : 1./n for k in num_outgoing.keys()}
    # zero list
    zeros_list = []
    for node,val in num_outgoing.items():
        if val == 0:
            zeros_list.append(node)
    import time
    for i in range(iters):
        # start time
        start = time.time()
        # gen is used for 
        gen = ((entries, pr, num_outgoing, d,zeros_list) for entries in grouper(incoming.items(), 2**11))
        # iterate to finish the iteration part
        pr = dict(mapreduce(pr_map, pr_reduce, gen, pool=pool))
        if i % 10 == 0:
            print("Time 10 iteration is: ", time.time() - start)
    return pr

Time 10 iteration is:  0.0003497600555419922
Time 10 iteration is:  0.0003027915954589844
Time 10 iteration is:  0.00030112266540527344
Time 10 iteration is:  0.00027489662170410156
Time 10 iteration is:  0.0002770423889160156
Time 10 iteration is:  0.0002720355987548828
Time 10 iteration is:  0.0002701282501220703
Time 10 iteration is:  0.0002701282501220703
Time 10 iteration is:  0.00027108192443847656
Time 10 iteration is:  0.0002739429473876953
Time 10 iteration is:  1.2932758331298828
Time 10 iteration is:  1.2870268821716309
Time 10 iteration is:  1.26214599609375
Time 10 iteration is:  1.3775677680969238
Time 10 iteration is:  1.3547208309173584
Time 10 iteration is:  1.3449671268463135
Time 10 iteration is:  1.2866466045379639
Time 10 iteration is:  1.3338561058044434
Time 10 iteration is:  1.3061189651489258
Time 10 iteration is:  1.4939980506896973
Completed pagerank in 128.2s
### TESTING pagerank: PASSED 15/16
# 15	: Assertion failed
###

